In [1]:

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from utils import read_diabetes_dataset
from imblearn.over_sampling import SMOTE

from utils import statistical_parity, average_odds, average_predictive_value, consistency, theil_index
from utils import Weighted, CounterfactualPreProcessing
from sklearn.linear_model import LogisticRegression
# from sklego.metrics import equal_opportunity_score
# from sklego.metrics import p_percent_score
from sklearn.metrics import accuracy_score



In [2]:
# data = pd.read_csv('diabetic_data.csv')
BINARY = True
X, y = read_diabetes_dataset(binary=BINARY)

X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=101)

/home/texs/anaconda3/envs/cuml-torch/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
clf_log = LogisticRegression(class_weight=None, max_iter=10**3)


clf_log.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [36]:
# sens_variable = 'race_Asian'
sens_variable = 'gender'
# Z_train = X_train['gender']
# Z_train = X_train['race_Caucasian']
# Z_train = X_train['race_AfricanAmerican']
# Z_train = X_train['race_Hispanic']
Z_train = X_train[sens_variable]



In [44]:

print(X_train.columns)
# print(np.inde)
sens_position = np.argwhere(X_train.columns==sens_variable)[0][0]

Index(['gender', 'age', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed',
       'race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic',
       'race_Other'],
      dtype='object')


In [45]:
y_train_ = clf_log.predict(X_train)

In [46]:
print('Accuracy', accuracy_score(y_train, y_train_))
print('Statistical parity', statistical_parity(y_train, y_train_, Z_train))
print('Average odds', average_odds(y_train, y_train_, Z_train))
print('Average predictive value',average_predictive_value(y_train, y_train_, Z_train))
# print('Consistency', consistency(X_train.values, y_train_, 5))
print('Theil index', theil_index(y_train, y_train_))

Accuracy 0.6258839039590908
Automatic priviledged value is 0
Statistical parity [-0.1005595]
Automatic priviledged value is 0
Average odds [-0.11236875]
Automatic priviledged value is 0
Average predictive value [-0.18907577]
Theil index 0.41179555755683683


In [47]:
sens_position

28

In [48]:
sensitive_feature_ids = [sens_position]

In [49]:
clf_fair_weighted = Weighted(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=sensitive_feature_ids)
clf_fair_weighted.fit(X_train.values, y_train)
y_train_ = clf_fair_weighted.predict(X_train.values)

/home/texs/Documentos/IC/TopicosIA/dataset_diabetes/utils.py:289: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  sensitive_columns_y = np.append(X[:, self.sensitive_feature_ids], y[:, np.newaxis], axis=1)


In [50]:
print('Accuracy', accuracy_score(y_train, y_train_))
print('Statistical parity', statistical_parity(y_train, y_train_, Z_train))
print('Average odds', average_odds(y_train, y_train_, Z_train))
print('Average predictive value',average_predictive_value(y_train, y_train_, Z_train))
# print('Consistency', consistency(X_train.values, y_train_, 5))
print('Theil index', theil_index(y_train, y_train_))

Accuracy 0.6257440773440933
Automatic priviledged value is 0
Statistical parity [-0.00475647]
Automatic priviledged value is 0
Average odds [-0.00426251]
Automatic priviledged value is 0
Average predictive value [-0.12908069]
Theil index 0.4122503848463734


In [51]:
clf_fair_counter = CounterfactualPreProcessing(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=sensitive_feature_ids)
clf_fair_counter.fit(X_train.values, y_train)
y_train_ = clf_fair_counter.predict(X_train.values)

(100124, 32) (100124,)


/home/texs/anaconda3/envs/cuml-torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [52]:
print('Accuracy', accuracy_score(y_train, y_train_))
print('Statistical parity', statistical_parity(y_train, y_train_, Z_train))
print('Average odds', average_odds(y_train, y_train_, Z_train))
print('Average predictive value',average_predictive_value(y_train, y_train_, Z_train))
# print('Consistency', consistency(X_train.values, y_train_, 5))
print('Theil index', theil_index(y_train, y_train_))

Accuracy 0.6257440773440933
Automatic priviledged value is 0
Statistical parity [-0.09140028]
Automatic priviledged value is 0
Average odds [-0.10297699]
Automatic priviledged value is 0
Average predictive value [-0.1827288]
Theil index 0.41220588996232793
